In [53]:
import numpy as np
import pandas as pd
import nltk
import pickle
import re


In [54]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\katiy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\katiy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
df=pd.read_csv('Final_IPC_label_data.csv')
df.head()

,Definition,Label
0,16 people gang rape and murder of a girl who w...,IPC_Section_302_376
1,2 constables had then brutally beaten person w...,IPC_Section_307
2,2 persons who kidnapped 2 girls to an old buil...,IPC_Section_376
3,2 youth raped the girl on the pretext of takin...,IPC_Section_376
4,3 men for gang-raping woman and attacked her w...,IPC_Section_307_376


In [56]:
df.shape

(578, 2)

In [57]:
corpus=[]
for i in range(0,df.shape[0]):
    definition=re.sub('[^a-zA-Z]',' ',df['Definition'][i])
    definition=definition.lower()
    definition=definition.split()
    wnlwm=nltk.WordNetLemmatizer()
    all_stopwords=stopwords.words('english')
    definition=[wnlwm.lemmatize(word) for word in definition if not word in set(all_stopwords)]
    definition=' '.join(definition)
    corpus.append(definition)

print (corpus)

['people gang rape murder girl burnt alive', 'constable brutally beaten person sustained injury admitted hospital critical condition', 'person kidnapped girl old building raped elder child', 'youth raped girl pretext taking nearby jungle', 'men gang raping woman attacked sharp weapon', 'men accused rape murder woman', 'men allegedly gang raped woman front husband', 'men raped girl strangulated death', 'men accused raping girl', 'case murder registered', 'driver allegedly killed old enmity arrested beaten people', 'group people allegedly attacked man fired shot', 'man axed death son', 'person intending victim poison procures white powder containing poison secretly mix food remains keeping person fails kill pioson viction', 'person said commit offense murder person sound mind kill reasonable person', 'position control dominance woman commits rape woman', 'relative guardian teacher person position trust authority towards woman commits rape woman', 'accused allegedly shot revolver allegedl

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=700)
X=cv.fit_transform(corpus).toarray()
y=df.iloc[:,1].values
print(X.shape)

(578, 683)


In [59]:
len(X[0])

683

In [60]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=0)
print(X_train.shape)

(491, 683)


In [78]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0,degree=5,tol=0.001,gamma='scale')
classifier.fit(X_train, y_train)
y_pred1 = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_307' 'IPC_Section_307']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_307_376' 'IPC_Section_307_376']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_307_376' 'IPC_Section_307_376']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_307_376' 'IPC_Section_307_376']
 ['IPC_Section_307_376' 'IPC_Section_307_376']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_302' 'IPC_Section_307']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_302_376' 'IPC_Section_307_376']
 ['IPC_Section_307' 'IPC_Section_307']
 ['IPC_Section_302' 'IPC_Section_302']
 ['IPC_Section_302_376' 'IPC_Section_302_376']
 ['IPC_Section_376' 'IPC_Section_302']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_376' 'IPC_Section_376']
 ['IPC_Section_3

In [79]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred1)

[[25  0  0  0  1]
 [ 0 10  0  0  3]
 [ 3  0  9  0  0]
 [ 0  1  0  5  5]
 [ 0  0  0  0 25]]


0.9310344827586207

In [83]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred1, average='macro')

0.9203596403596404

In [85]:
with open("svm_model.pkl", "wb") as file:
    pickle.dump(classifier, file)

